In [6]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
import librosa

In [7]:

# Load your dataset
df = pd.read_csv(r"C:\Users\user\Desktop\Project\EXPERIMENTS\revised-data\audio_features_cp.csv")

# Define feature columns and target column
x_cols = ['mfcc_1', 'mfcc_2', 'mfcc_3', 'mfcc_4', 'mfcc_5', 'mfcc_6', 'mfcc_7', 'mfcc_8', 
           'mfcc_9', 'mfcc_10', 'mfcc_11', 'mfcc_12', 'mfcc_13', 'spectral_centroid', 
           'spectral_bandwidth', 'zero_crossing_rate' , 'spectrogram_mean' , 'spectrogram_median' , 'spectrogram_variance']
y_cols = ['label']

# Normalize the feature data to range [0,1] using MinMaxScaler
SMM = MinMaxScaler(feature_range=(0, 1))
df[x_cols] = SMM.fit_transform(df[x_cols])

# Split data into features and labels
X = df[x_cols].values  # Convert to NumPy array
y = df[y_cols].values.ravel()  # Convert to 1D array

# Train-test split (20% test set)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape the data for RNN (samples, timesteps, features) where timesteps=1 since it's not a time series
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])



In [32]:
# Define the RNN model
model = Sequential()

# Add the RNN layer (input_shape is (timesteps, features) - timesteps=1 and features=19)
model.add(SimpleRNN(50, input_shape=(1, 19), activation='relu'))

# Add dropout for regularization
model.add(Dropout(0.3))

# Add output layer for binary classification (sigmoid for binary classification)
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Summarize the model architecture
model.summary()


C:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ simple_rnn_2 (SimpleRNN)             │ (None, 50)                  │           3,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 50)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              51 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,551 (13.87 KB)

 Trainable params: 3,551 (13.87 KB)

 Non-trainable params: 0 (0.00 B)

In [33]:

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))



Epoch 1/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.5405 - loss: 0.6883 - val_accuracy: 0.7152 - val_loss: 0.6536
Epoch 2/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6613 - loss: 0.6507 - val_accuracy: 0.7803 - val_loss: 0.6174
Epoch 3/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7031 - loss: 0.6256 - val_accuracy: 0.7848 - val_loss: 0.5850
Epoch 4/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7232 - loss: 0.5931 - val_accuracy: 0.8091 - val_loss: 0.5508
Epoch 5/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7855 - loss: 0.5604 - val_accuracy: 0.8227 - val_loss: 0.5217
Epoch 6/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7856 - loss: 0.5362 - val_accuracy: 0.8333 - val_loss: 0.4940
Epoch 7/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7986 - loss: 0.5167 - val_accuracy: 0.8379 - val_loss: 0.4712
Epoch 8/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8042 - loss: 0.4935 - val_accuracy: 0.8242 - val_loss

In [34]:
### Evaluation Metrics
# Make predictions on the test set
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype("int32")

# Evaluate metrics
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1}")

conf_matrix = confusion_matrix(y_test, y_pred)
print(f"Confusion Matrix:\n{conf_matrix}")

class_report = classification_report(y_test, y_pred)
print(f"Classification Report:\n{class_report}")


21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step
Accuracy: 0.8924242424242425
F1 Score: 0.897841726618705
Confusion Matrix:
[[277  37]
 [ 34 312]]
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.88      0.89       314
           1       0.89      0.90      0.90       346

    accuracy                           0.89       660
   macro avg       0.89      0.89      0.89       660
weighted avg       0.89      0.89      0.89       660



In [35]:

### Prediction Function for New Audio
def extract_features(audio_file):
    # Load the audio file
    y, sr = librosa.load(audio_file, sr=None)
    
    # Extract MFCC (first 13 coefficients)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    mfccs_mean = np.mean(mfccs.T, axis=0)
    
    # Extract Spectral Centroid
    spectral_centroid = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
    
    # Extract Spectral Bandwidth
    spectral_bandwidth = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))
    
    # Extract Zero-Crossing Rate
    zero_crossing_rate = np.mean(librosa.feature.zero_crossing_rate(y=y))
    
    # Extract Spectrogram (STFT)
    spectrogram = np.abs(librosa.stft(y))
    spectrogram_mean = np.mean(spectrogram.T, axis=0)  # Get mean of spectrogram along time axis
    spectrogram_mean2 = np.mean(spectrogram_mean)
    spectrogram_median = np.median(spectrogram_mean)
    spectrogram_variance = np.var(spectrogram_mean)
    
    # Return the features as a list (order must match the training features)
    return [
        mfccs_mean[0], mfccs_mean[1], mfccs_mean[2], mfccs_mean[3], mfccs_mean[4],
        mfccs_mean[5], mfccs_mean[6], mfccs_mean[7], mfccs_mean[8], mfccs_mean[9],
        mfccs_mean[10], mfccs_mean[11], mfccs_mean[12],
        spectral_centroid, spectral_bandwidth, zero_crossing_rate,
        spectrogram_mean2, spectrogram_median, spectrogram_variance
    ]

# Function to predict new audio file
def pred(audio_file):
    # Extract features from the audio file
    features = extract_features(audio_file)
    
    # Convert the features to NumPy array and reshape for RNN
    feature_array = np.array(features).reshape(1, 1, -1)  # Reshape for RNN (1 sample, 1 timestep, 19 features)
    
    # Scale the features using the previously fitted scaler
    feature_array = SMM.transform(feature_array.reshape(1, -1)).reshape(1, 1, -1)  # Scale and reshape back for RNN
    
    # Predict using the trained model
    prediction_prob = model.predict(feature_array)
    prediction = (prediction_prob > 0.5).astype("int32")
    
    if prediction == 1:
        print("Screaming")
    else:
        print("Non_screaming")
    
    print(f"Prediction Probability: {prediction_prob}")



In [40]:
# Test the prediction on a new audio file
ipt = r"C:\Users\user\Desktop\Project\EXPERIMENTS\testaudio\test5.wav"
pred(ipt)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Non_screaming
Prediction Probability: [[0.29566067]]


C:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
